In [1]:
from ortools.linear_solver import pywraplp
import numpy as np

In [37]:
def modelo_master(M, R, Q, P, scores, m, r, q, p, N, model=1):
    """
    pre:
        ya están ordenados los chabones por score decreciente
    """
    
    K = len(scores)
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Variables
    x = {}
    for i in range(K):
        x[i] = solver.BoolVar(f'x[{i}]')

    # Restricciones
    solver.Add(solver.Sum([x[i] for i in range(K)]) == N)
    solver.Add(solver.Sum([x[i] for i in M.nonzero()[0]]) >= m * N/100)
    solver.Add(solver.Sum([x[i] for i in R.nonzero()[0]]) >= r * N/100)
    solver.Add(solver.Sum([x[i] for i in Q.nonzero()[0]]) >= q * N/100)
    solver.Add(solver.Sum([x[i] for i in P.nonzero()[0]]) >= p * N/100)

    if model == 1:
        solver.Maximize(solver.Sum([scores[i] * x[i] for i in range(K)]))
    else:
        solver.Minimize(solver.Sum([ i * x[i] for i in range(K)]))

    return solver, x

In [38]:
# Total
K = 200

# Porcentajes de admisión
m = 30
r = 30
q = 20
p = 20

In [39]:
def generar_instancia(K, m_pob = 0.5, r_pob = 0.1, q_pob = 0.4, p_pob = 0.2):
    M = np.random.uniform(0, 1, K) < m_pob
    R = np.random.uniform(0, 1, K) < r_pob
    Q = np.random.uniform(0, 1, K) < q_pob
    P = np.random.uniform(0, 1, K) < p_pob
    scores = np.random.rand(K)*100
    scores = np.sort(scores)[::-1]
    return M, R, Q, P, scores

In [40]:
solver, x = modelo_master(*generar_instancia(K), m, r, q, p, 20, 1 )

In [41]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    print("Objective value =", solver.Objective().Value())
    for t in range(K):
        if x[t].solution_value() != 0:
            print(f"x[{t}] =", x[t].solution_value())
else:
    print("The problem does not have an optimal solution.")

print("\nAdvanced usage:")
print(f"Problem solved in {solver.wall_time():d} milliseconds")
print(f"Problem solved in {solver.iterations():d} iterations")
print(f"Problem solved in {solver.nodes():d} branch-and-bound nodes")

Solving with Cbc 2.10.7
Solution:
Objective value = 1838.156988365055
x[0] = 1.0
x[1] = 1.0
x[2] = 1.0
x[3] = 1.0
x[4] = 1.0
x[5] = 1.0
x[6] = 1.0
x[7] = 1.0
x[8] = 1.0
x[9] = 1.0
x[10] = 1.0
x[11] = 1.0
x[12] = 1.0
x[13] = 1.0
x[14] = 1.0
x[15] = 1.0
x[16] = 1.0
x[19] = 1.0
x[45] = 1.0
x[53] = 1.0

Advanced usage:
Problem solved in 1500 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes
